<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/Crear_DEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

# Celda Final: Autenticación, Inicialización y Prueba de Uso

import ee
from IPython.display import display, Markdown

# ⚠️ Usamos tu ID de proyecto.
PROYECTO_ID = "gen-lang-client-0528470346"

# 1. FORZAR LA AUTENTICACIÓN (pedirá un nuevo enlace y código)
display(Markdown("### ⚠️ Paso 1: Forzar Autenticación"))
display(Markdown("Si no aparece el enlace, reinicia e inténtalo de nuevo."))

# El comando 'ee.Authenticate' debería mostrar el enlace y pedir el código.
ee.Authenticate()

# 2. Inicialización
try:
    display(Markdown("### ✅ Paso 2: Inicializando con Proyecto..."))
    ee.Initialize(project=PROYECTO_ID)

    # Prueba rápida para confirmar que GEE está listo
    test_geometry = ee.Geometry.Point(-71.146, 8.587)
    display(Markdown("✅ **¡ÉXITO! Google Earth Engine está listo para usar.**"))

except Exception as e:
    display(Markdown(f"❌ **Error Fatal:** La inicialización falló. El problema debe ser un permiso faltante en el proyecto **`{PROYECTO_ID}`** dentro de la consola de Google Cloud, a pesar de que la API esté habilitada."))

### ⚠️ Paso 1: Forzar Autenticación

Si no aparece el enlace, reinicia e inténtalo de nuevo.

### ✅ Paso 2: Inicializando con Proyecto...

✅ **¡ÉXITO! Google Earth Engine está listo para usar.**

In [5]:

import folium
import ee.mapclient # Para usar ee.mapclient.addTo(map)

In [6]:

# 1. Definir el centro de la región de interés (ejemplo: un punto en los Andes)
latitud = 8.587  # Ejemplo de Mérida, Venezuela
longitud = -71.146
centro = ee.Geometry.Point(longitud, latitud)

# 2. Cargar el conjunto de datos SRTM DEM
# El DEM de SRTM de 30 metros de resolución
dataset = ee.Image('USGS/SRTMGL1_003')

# 3. Seleccionar la banda de elevación
elevacion = dataset.select('elevation')

# 4. Definir los parámetros de visualización
# Rango de elevación común para una buena visualización (ejemplo: 0 a 4000 metros)
vis_params = {
  'min': 0,
  'max': 5000,
  'palette': ['006600', '99ff00', 'cc0000', 'ffffff'] # Colores: verde, amarillo, rojo, blanco
}

# 5. Obtener el valor de la elevación en el punto central (prueba)
elevacion_media = elevacion.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=centro.buffer(500), # Área de 500m alrededor del punto
    scale=30
).get('elevation').getInfo()

print(f"✅ Elevación media cerca del punto central: {elevacion_media:.2f} metros")

✅ Elevación media cerca del punto central: 1433.28 metros


In [7]:

## 🌍 Configurar el Mapa Base

# 1. Crear un mapa base centrado en la región
mapa_ee = folium.Map(location=[latitud, longitud], zoom_start=10)

# 2. Agregar la capa del DEM al mapa
# La función 'getMapId()' prepara la imagen para ser visualizada como un mosaico web
mapa_info = elevacion.getMapId(vis_params)
tile_url = mapa_info['tile_fetcher'].url_format

# 3. Agregar el TileLayer (mosaico) de Earth Engine a Folium
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    name='SRTM Elevation',
    overlay=True,
    control=True
).add_to(mapa_ee)

# 4. Agregar controles para alternar capas
folium.LayerControl().add_to(mapa_ee)

# 5. Mostrar el mapa interactivo
display(mapa_ee)